# Tracking with Trackastra

In [1]:
import torch
from trackastra.model import Trackastra
from trackastra.tracking import graph_to_ctc, graph_to_napari_tracks
from pathlib import Path
from tifffile import imread
import skimage as ski
import numpy as np

INFO:numexpr.utils:Note: detected 72 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
INFO:numexpr.utils:Note: NumExpr detected 72 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# load some test data images and masks
root_path = Path("../../pvc/scratch/interaction_cells/datasets/")
print("Loading image...")
imgs = imread(root_path / "series003_cCAR_tumor.tif")
print("Loaded image")
imgs = imgs[:, 0, ...]

Loading image...
Loaded image


In [3]:
imgs = ski.transform.resize(imgs, (imgs.shape[0], imgs.shape[1] // 2, imgs.shape[2] // 2), anti_aliasing=True)

In [4]:
masks = imread("../../pvc/scratch/SHARE/cancer_cell_masks/series003_cCAR_tumor.tif").astype(np.uint16)

In [5]:
# Load a pretrained model
model = Trackastra.from_pretrained("general_2d", device=device)

INFO:trackastra.model.model:Loading model state from /home/achard/.trackastra/.models/general_2d/model.pt
INFO:trackastra.model.model_api:Using device cuda


/home/achard/.trackastra/.models/general_2d already downloaded, skipping.


In [10]:
# Track the cells
track_graph = model.track(imgs, masks.astype(np.uint16), mode="ilp", use_distance=True, max_distance=30, allow_divisions=False)  # or mode="ilp", or "greedy_nodiv"

INFO:trackastra.model.model_api:Predicting weights for candidate graph
INFO:trackastra.data.wrfeat:Extracting features from 162 detections
INFO:trackastra.data.wrfeat:Using single process for feature extraction
Extracting features: 100%|██████████| 162/162 [00:12<00:00, 12.50it/s]
INFO:trackastra.model.model_api:Building windows
Building windows: 100%|██████████| 159/159 [00:00<00:00, 14996.84it/s]
INFO:trackastra.model.model_api:Predicting windows
Computing associations: 100%|██████████| 159/159 [00:05<00:00, 29.77it/s]
INFO:trackastra.model.model_api:Running greedy tracker
INFO:trackastra.tracking.tracking:Build candidate graph with delta_t=1
INFO:trackastra.tracking.tracking:Added 26729 vertices, 32401 edges                          
INFO:trackastra.tracking.ilp:Using `gt` ILP config.
INFO:motile.solver:Adding NodeSelection cost...
INFO:motile.solver:Adding NodeSelected variables...
INFO:motile.solver:Adding EdgeSelection cost...
INFO:motile.solver:Adding EdgeSelected variables...
I


Candidate graph		26729 nodes	32401 edges
Solution graph		26729 nodes	25316 edges


In [11]:
# Write to cell tracking challenge format
ctc_tracks, masks_tracked = graph_to_ctc(
      track_graph,
      masks,
      outdir="../../pvc/scratch/SHARE/cancer_cell_masks/series003_cCAR_tumor_tracked",
)

Saving masks: 100%|██████████| 162/162 [00:00<00:00, 181.85it/s]
